In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import googlemaps
import os
import datetime

import funciones
from shapely.geometry import Point

import matplotlib.pyplot as plt

#para decodificar polilineas
#https://developers.google.com/maps/documentation/utilities/polylineutility
#'`__rEb|ucJl@cB|BgGbOa\\bHuQh@cB@CdBmFnCcH~BgM~A}GbBcIh@qBCJ'

#para volver a leer dict de texto
import ast


In [2]:
#Creo la geometria
geo = gpd.read_file('geometria/villas_asentamientosPUNTOS_AGBA_posgar5.shp') 
#cambio coordenadas a latlong para google
geo = geo.to_crs(epsg=4326)
geo['X'] = [geo.geometry.iloc[i].centroid.x for i in range(geo.shape[0])]
geo['Y'] = [geo.geometry.iloc[i].centroid.y for i in range(geo.shape[0])]


In [17]:
variablesMapeo = [ u'tramos',u'distancia',u'costo',u'tiempoTotal','descripcion',
              u'duracionIngreso', u'distanciaIngreso',
         u'duracionEgreso',  u'distanciaEgreso',  u'tiempoEnSistema']

#variables con problemas ascii 
variablesAsciiProblemas = [u'dpto',
                           u'munic',
                           u'nombre',
                           u'obs_otros',
                          ]

In [7]:
#creo el vector de destinos
#ver si invertir
destinos = [(-34.605655, -58.374446)]


## Mejor viaje

In [8]:
bestTrip = funciones.generarConsulta(geo.copy(),destinos[0],transit_routing_preference = False)

In [9]:
#Sin respuesta
sum([len(bestTrip.consulta.iloc[i])== 0 for i in range(bestTrip.shape[0])])

39

In [ ]:
#guardar en csv sin variabls ascii

In [11]:
geo.columns

Index([  u'ORIG_FID',       u'dpto', u'familias_p',   u'geometry',
           u'id_PBA',   u'idtrucho',      u'munic',     u'nombre',
        u'obs_otros',   u'orig_pba',     u'origen',       u'prov',
         u'tipo_nos',   u'tipo_ofi',          u'X',          u'Y'],
      dtype='object')

In [37]:
bestTrip.drop(variablesAsciiProblemas,axis=1).to_csv('bestTrip.csv',index=False)

In [ ]:
#dataBacup = pd.read_csv('bestTrip.csv')
#dataBacup = dataBacup.consulta.map(ast.literal_eval)


In [41]:
bestTripConDatos = funciones.extraerDataConsulta(data=bestTrip,
                                      destino = destinos[0],
                                      transit_routing_preference = 'mejor viaje') 

In [42]:
bestTripConDatos2 = funciones.generarDistancia(bestTripConDatos,destinos[0])

In [43]:
bestTripConDatos2.drop(variablesAsciiProblemas,axis=1,inplace=True)

In [44]:
nombre='bestTripConDatos2'
bestTripConDatos2.descripcion = bestTripConDatos2.descripcion.map(str)
bestTripConDatos2.modos = bestTripConDatos2.modos.map(str)
bestTripConDatos2.consulta = bestTripConDatos2.consulta.map(str)

    
bestTripConDatos2.to_file('resultados/'+str(nombre))
bestTripConDatos2.to_csv('resultados/'+str(nombre)+'/'+str(nombre)+'.csv')

## Menos trasbordos

In [38]:
menosTrans = funciones.generarConsulta(geo.copy(),destinos[0],
                                             transit_routing_preference = 'fewer_transfers')

In [39]:
menosTrans.drop(variablesAsciiProblemas,axis=1).to_csv('menosTrans.csv',index=False)

In [45]:
menosTransConData = funciones.extraerDataConsulta(data=menosTrans,
                                      destino = destinos[0],
                                      transit_routing_preference = 'menos transbordos') 

In [46]:
menosTransConData2 = funciones.generarDistancia(menosTransConData,destinos[0])

In [47]:
menosTransConData2.drop(variablesAsciiProblemas,axis=1,inplace=True)

In [48]:
nombre='menosTransConData'
menosTransConData2.descripcion = menosTransConData2.descripcion.map(str)
menosTransConData2.modos = menosTransConData2.modos.map(str)
menosTransConData2.consulta = menosTransConData2.consulta.map(str)

    
menosTransConData2.to_file('resultados/'+str(nombre))
menosTransConData2.to_csv('resultados/'+str(nombre)+'/'+str(nombre)+'.csv')